# Mini-lab 1: Zillow Dataset Logistic Regression and SVMs  
MSDS 7331 Data Mining - Section 403 - Mini Lab 1

Team: Ivelin Angelov, Yao Yao, Kaitlin Kirasich, Albert Asuncion

<a id="top"></a>
## Contents
* <a href="#Imports">Imports</a>
* <a href="#Models">Models</a>
* <a href="#Advantages">Advantages of Each Model</a>
* <a href="#Feature">Feature Importance</a>
* <a href="#Insights">Insights</a>
________________________________________________________________________________________________________

<a id="Imports"></a>

# Imports
We chose to use the same Zillow dataset from Lab 1. For origin and purpose of dataset as well as a detailed description of the dataset, refer to https://github.com/post2web/data_mining_group_project/blob/master/notebooks/lab1.ipynb.

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')

# Load Data, Create y and X
We will use the Zillow dataset from our previous lab, where the cleanup files were exported from lab 1 into mini-lab 1. Note that for logistic regression and support vector classifier models, we choose to use mostly complete continuous variables as well as create dummy variables for nominal variables to cross compare the performance, feature importance, and insights of each model. X is the training set and y is the test set, where we are testing if our models can accurately predict positive logerrror from that of negative. 

Data columns that are only available for the training set and not the test set (transaction date) were removed. Parcelid was removed because each individual property has its own ID and does not correlate well with regression or SVMs. The column that was created for "New Features" from Lab 1 (city and pricepersqft) were also removed for the sake of simplicity of only using original data for the prediction process.

In [2]:
# load datasets here:
variables = pd.read_csv('../datasets/variables.csv').set_index('name')
X = pd.read_csv('../datasets/train.csv', low_memory=False)

y = (X['logerror'] > 0).astype(np.int32)

del X['logerror']
del X['transactiondate']
del X['parcelid']
del X['city']
# TODO fix me
del X['price_per_sqft']

'The dataset has %d rows and %d columns' % X.shape

'The dataset has 116761 rows and 49 columns'

In [3]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116761 entries, 0 to 116760
Data columns (total 49 columns):
Unnamed: 0                      116761 non-null int64
airconditioningtypeid           116761 non-null int64
bathroomcnt                     116761 non-null float64
bedroomcnt                      116761 non-null int64
buildingqualitytypeid           116761 non-null int64
calculatedbathnbr               116761 non-null float64
calculatedfinishedsquarefeet    116761 non-null int64
finishedsquarefeet12            116761 non-null int64
finishedsquarefeet50            116761 non-null float64
fips                            116761 non-null int64
fireplacecnt                    116761 non-null int64
fullbathcnt                     116761 non-null float64
garagecarcnt                    116761 non-null float64
garagetotalsqft                 116761 non-null float64
hashottuborspa                  116761 non-null int64
heatingorsystemtypeid           116761 non-null int64
latitude     

# Dealing with Nominal Data
Nominal data ususally has more than two values. For logistic regression and SVMs, we created dummy variables that only factor in 0s and 1s for the prediction process of logistic regression and SVMs.

In [4]:
nominal = variables[variables['type'].isin(['nominal'])]
nominal = nominal[nominal.index.isin(X.columns)]
nominal_data = X[nominal.index]

nominal_data = pd.get_dummies(nominal_data, drop_first=True)
nominal_data = nominal_data[nominal_data.columns[~nominal_data.columns.isin(nominal.index)]]

# Dealing with Continuous Data
StandardScaler from sklean was applied to the continuous data columns to standardize the dataset around center 0 with equal variance for creating normal distribtions prior to the application of logistic regressio and SVMs.

In [5]:
from sklearn.preprocessing import StandardScaler

continuous = variables[~variables['type'].isin(['nominal'])]
continuous = continuous[continuous.index.isin(X.columns)]

X = X[continuous.index]
columns = X.columns

# X = (X - µ) / σ

scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

X = pd.DataFrame(X, columns=columns)

In [6]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116761 entries, 0 to 116760
Data columns (total 30 columns):
bathroomcnt                     116761 non-null float64
bedroomcnt                      116761 non-null float64
buildingqualitytypeid           116761 non-null float64
calculatedbathnbr               116761 non-null float64
calculatedfinishedsquarefeet    116761 non-null float64
finishedsquarefeet12            116761 non-null float64
finishedsquarefeet50            116761 non-null float64
fireplacecnt                    116761 non-null float64
fullbathcnt                     116761 non-null float64
garagecarcnt                    116761 non-null float64
garagetotalsqft                 116761 non-null float64
hashottuborspa                  116761 non-null float64
latitude                        116761 non-null float64
longitude                       116761 non-null float64
lotsizesquarefeet               116761 non-null float64
poolcnt                         116761 non-null f

## Merge data
The data was then merged for the application of logistic regression and SVM prediction. The following shows the final shape of the dataset after the application of dummy variables and StandardScaler.

In [7]:
X = pd.concat([X, nominal_data], axis=1)
'The dataset has %d rows and %d columns' % X.shape

'The dataset has 116761 rows and 2107 columns'

In [8]:
X.loc[1,'bathroomcnt']

-1.2777713394270958

<a id="Models"></a>
<a href="#top">Back to Top</a>
# Models

<b>[50 points]</b>

<i>
Create a logistic regression model and a support vector machine model for the classification task involved with your dataset. Assess how well each model performs (use 80/20 training/testing split for your data). <b>Adjust parameters of the models to make them more accurate</b>. If your dataset size requires the use of stochastic gradient descent, then linear kernel only is fine to use. That is, the SGDClassifier is fine to use for optimizing logistic regression and linear support vector machines. For many problems, SGD will be required in order to train the SVM model in a reasonable timeframe. 
</i>

## Logistic Regression
For the logistic regression model, we created a function that took in X_train and Y_train from the original data set to test for X_test from the modified dataset. The accuracy of the logistic regression prediction for positive or negative logerror was compared with that of the original, where a confusion matrix was made to show percentage accuracy. Due to the complexity of the dataset, we are slightly better than 50% accuracy.

In [ ]:
def logistic_regression_model(X_train, y_train, X_test, **params):
    clf = LogisticRegression(**params)
    clf.fit(X_train, y_train)
    return clf.predict(X_test), clf

yhat, _ = logistic_regression_model(X_train=X, y_train=y, X_test=X)

accuracy = float(sum(yhat==y)) / len(y)
print('Percentage accuracy: %.12f' % (accuracy))

pd.DataFrame(confusion_matrix(y, yhat),
    columns=['Predicted 1', 'Predicted 0'], 
    index=['Actual 1', 'Actual 0'], 
)

By running logistic regression one time with the built in parameters, we got an accuracy of 0.59.  To try to improve this, we want to do a few things. 

First, we want to do the 80/20 split 5 times and average those results to get a better accuracy.  By splitting the training and test sets up multiple times, we can minimize the effects of outliers.

Second, we want to see how changing the value of C will effect the accuracy.  To do this we have another for loop which sets C at 10 linear increments from 0.01 to 10.

In [ ]:
n_splits = 5           # number of times we will split test and training dataset
accuracies = []        # array to hold accuracy to take average of at the end
final_accuracies = {}  # list of final accuracies

# Now for each value of C and for each train/ test split run logistic regression
for C in np.linspace(.01, 50, 20):
    for i in range(n_splits):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

        yhat, _ = logistic_regression_model(
            X_train=X_train,
            y_train=y_train,
            X_test=X_test,
            C=C
        )
        
        accuracy = float(sum(yhat==y_test)) / len(y_test)
        accuracies.append(accuracy)

    print('Final average accuracy: %.12f' % (np.mean(accuracies)))
    final_accuracies[C] = np.mean(accuracies)
    
final_accuracies = pd.Series(final_accuracies)

# Plot the final accuracy for each value of C
plt.title('Accuracy of Logistic Regression Models Based on C Values', fontsize=15)
plt.xlabel('C Values', fontsize=15)
plt.ylabel('Accuracy', fontsize=15)

fig, ax = plt.subplots(figsize=(15,10))

final_accuracies.plot(ax=ax);

Final average accuracy: 0.568663555004
Final average accuracy: 0.569006123410
Final average accuracy: 0.569337272870
Final average accuracy: 0.569320858134
Final average accuracy: 0.569612469490
Final average accuracy: 0.569992434948
Final average accuracy: 0.569932281567
Final average accuracy: 0.570155868625
Final average accuracy: 0.570145163362
Final average accuracy: 0.570088639575
Final average accuracy: 0.570094556666
Final average accuracy: 0.570080931786
Final average accuracy: 0.569979149442
Final average accuracy: 0.569938398860
Final average accuracy: 0.570014987368


# L1 VS L2

We further compared accuracy based on L1 or L2 penalty, where we found L1, the error, is slightly more accurate than L2, the squared error. Which is interesting because typically L2 is better for minimizing error than L1. We think this is because we have about 2000 variables and did not go through and check if every one made sense.  Since L1 acts as a built in feature selection by giving weights nearly 0, this could be why our L1 is better than L2.  This is also a concern for overfitting the model but we will continue as our metric goal here is to maximize accuracy.

In [ ]:
n_splits = 2
accuracies = []
final_accuracies = {}

for penalty in ['l1', 'l2']:
    for i in range(n_splits):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

        yhat, _ = logistic_regression_model(
            X_train=X_train,
            y_train=y_train,
            X_test=X_test,
            penalty=penalty
        )
        
        accuracy = float(sum(yhat==y_test)) / len(y_test)
        accuracies.append(accuracy)

    print('Final average accuracy for %s : %.12f' % (penalty, np.mean(accuracies)))
    final_accuracies[penalty] = np.mean(accuracies)
    
final_accuracies = pd.Series(final_accuracies)

plt.title('Accuracy of Logistic Regression Models Based on Penalty Type', fontsize=15)
plt.xlabel('Penalty Type', fontsize=15)
plt.ylabel('Accuracy', fontsize=15)


fig, ax = plt.subplots(figsize=(15,10))

final_accuracies.plot(kind='bar', ax=ax);

## Support Vector Machine Classifier
For the support vector machine model, we created a function that took in X_train and Y_train from the original data set to test for X_test from the modified dataset. The accuracy of the SVM prediction for positive or negative logerror was compared with that of the original, where a confusion matrix was made to show percentage accuracy. Due to the complexity of the dataset, we are slightly better than 50% accuracy.

## Why we use SGDClassifier?

We tried out a few sklearn support vector machine functions and noticed that the accuracy was similar for each but with such a large dataset we decided to use time as our performance metric.  

First first tried SVC setting kernal = 'linear' but waited a long time for it to finish.
Next, we tried LinearSVC because the liblinear library it uses tends to be faster to converge the larger the number of samples is than the libsvm library.  

Lastly, we tried and found our winner, SGDClassifier with loss = 'hinge' to use a stochastic gradient descent, which was exponentially faster than the others because it only uses a subset of the dataset.

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC, LinearSVC

def support_vector_machine_model(X_train, y_train, X_test, **params):
    # takes about 1 minute
    params['n_iter'] = 1
    params['loss'] = 'hinge'
    params['alpha'] = 0.9
    params['epsilon'] = 0.00078
    clf = SGDClassifier(**params)
    # this takes too long
    #clf = SVC(kernel='linear')
    #clf = LinearSVC(**params)
    clf.fit(X_train, y_train)
    return clf.predict(X_test), clf

yhat, _ = support_vector_machine_model(X_train=X, y_train=y, X_test=X)

accuracy = float(sum(yhat==y)) / len(y)
print('Accuracy: %.12f' % (accuracy))

pd.DataFrame(confusion_matrix(y, yhat),
    columns=['Predicted 1', 'Predicted 0'], 
    index=['Actual 1', 'Actual 0'], 
)

# to do: evaluate performance for SVM

In [ ]:
n_splits = 2           # number of times we will split test and training dataset
accuracies = []        # array to hold accuracy to take average of at the end
final_accuracies = {}  # list of final accuracies

# Now for each value of C and for each train/ test split run logistic regression
for testable in np.linspace(.00001, .01, 3):
    for i in range(n_splits):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

        yhat, _ = support_vector_machine_model(
            X_train=X_train,
            y_train=y_train,
            X_test=X_test,
            alpha=testable
        )
        
        accuracy = float(sum(yhat==y_test)) / len(y_test)
        accuracies.append(accuracy)

    print('Final average accuracy: %.12f' % (np.mean(accuracies)))
    final_accuracies[testable] = np.mean(accuracies)
    
final_accuracies = pd.Series(final_accuracies)

# Plot the final accuracy for each value of C
plt.title('Accuracy of CVM Models Based on Alpha Values', fontsize=15)
plt.xlabel('Alpha Values', fontsize=15)
plt.ylabel('Accuracy', fontsize=15)

fig, ax = plt.subplots(figsize=(15,10))

final_accuracies.plot(ax=ax);

# TODO same with epsilon

# TODO same with l1 and l2

# Compare

In [ ]:
data = [[.55, 2000], [.57, 4000]]
table = pd.DataFrame(data, columns=['Accuracy', 'Time it takes'], index=['Logistic Regression', 'Support Vecotr Machine'])
print('Compare result ot the two models')
table

<a id="Advantages"></a>
<a href="#top">Back to Top</a>
## Advantages of Each Model
<b>[10 points]</b>

<i>
Discuss the advantages of each model for each classification task. Does one type of model offer superior performance over another in terms of prediction accuracy? In terms of training time or efficiency? Explain in detail. 
</i>

# to do: write advantage after SVM performance parameters are found

<a id="Feature"></a>
<a href="#top">Back to Top</a>
## Feature Importance
<b>[30 points]</b>
    
<i>
Use the weights from logistic regression to interpret the importance of different features for the classification task. Explain your interpretation in detail. Why do you think some variables are more important?
</i>

# Logistic Regression Feature Importance

After scaling the continuous variables, the feature importance showed that propertyzoningdesc and propertycountylandusecode for multiple counties showed the most importance for predicting logerror. This might be due to the fact that neighborhoods based on location highly dictate the sales price of a property. Tax was also a big factor. Perhaps the more land owners pay for property tax could better predict property value because more amenities could be added for a richer neighborhood than that for a poorer.

In [ ]:
_, clf = logistic_regression_model(X_train=X, y_train=y, X_test=X, C=2)

index = pd.Index(X.columns, name='Variable Name')

importance = pd.Series(np.abs(clf.coef_[0]), index=index)
importance = importance.sort_values()

importance = importance.iloc[:50]

fig, ax = plt.subplots(figsize=(15, 20))
importance.plot(kind='barh', ax=ax)
plt.title('Logistic Regression Feature Importance (TOP 50 Variables)')
plt.xlabel('Importance', fontsize=8)
plt.ylabel('Feature', fontsize=15);

<a id="Insights"></a>
<a href="#top">Back to Top</a>
## Insights
<b>[10 points]</b>

<i>
Look at the chosen support vectors for the classification task. Do these provide any insight into the data? Explain. If you used stochastic gradient descent (and therefore did not explicitly solve for support vectors), try subsampling your data to train the SVC model — then analyze the support vectors from the subsampled dataset. 
</i>

# Insights from SVM Model

We used support vectors for gradient descent to find the optimal accuracy of the SVM model.

In [ ]:
from sklearn.svm import SVC
clf = SVC(kernel='linear', max_iter=10)
clf.fit(X, y)

# this hold the indexes of the support vectors
clf.support_

# this holds a subset of the data which is used for support vectors
clf.support_vectors_

# get number of support vectors for each class
print('Nubmer of support vectros for each feature:', clf.n_support_)

In [ ]:
clf.support_

In [ ]:
clf.support_vectors_

In [ ]:
df_clf_support = pd.DataFrame(clf.support_)

In [ ]:
df_clf_support

In [ ]:
df_clf_supportvectors = pd.DataFrame(clf.support_vectors_)

In [ ]:
#df_clf_supportvectors

In [ ]:
df_clf_supportvectors['logerror'] = y[clf.support_vectors_]

<a href="#top">Back to Top</a>
# References:

- Kernels from Kaggle competition: https://www.kaggle.com/c/zillow-prize-1/kernels
- Scikitlearn logistic regression: http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
- Scikitlearn linear SVC: http://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html
- Stackoverflow pandas questions: https://stackoverflow.com/questions/tagged/pandas